# Exercise 04: 2D Grid Indexing 📊

Work with 2D data (images, matrices) using 2D thread grids!

## Learning Goals
- Use `dim3` for 2D block and grid dimensions
- Calculate 2D thread indices (row, col)
- Understand row-major memory layout
- Process matrices in parallel

## 🚀 Setup

**Enable GPU**: Runtime → Change runtime type → T4 GPU → Save

In [ ]:
!nvcc --version
!nvidia-smi --query-gpu=name --format=csv,noheader

## 📚 Key Concepts

### 2D Launch Configuration with dim3
```cpp
dim3 threadsPerBlock(16, 16);  // 16x16 = 256 threads
dim3 blocksPerGrid(
    (width + 15) / 16,   // Columns (x)
    (height + 15) / 16   // Rows (y)
);
kernel<<<blocksPerGrid, threadsPerBlock>>>(...);
```

### 2D Thread Indexing
```cpp
int col = blockIdx.x * blockDim.x + threadIdx.x;  // X → column
int row = blockIdx.y * blockDim.y + threadIdx.y;  // Y → row
```

### Row-Major Memory Layout
```
Matrix A[3][4]:           Stored in 1D memory:
┌────────────────┐        
│ 0  1  2  3 │ Row 0     [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
│ 4  5  6  7 │ Row 1      ↑           ↑           ↑
│ 8  9 10 11 │ Row 2      Row 0       Row 1       Row 2
└────────────────┘

index = row * width + col
Example: A[1][2] → 1 * 4 + 2 = 6
```

### Visual: Thread to Matrix Mapping
```
Matrix 64x48        Grid of Blocks (4x3)     Each Block (16x16)
┌─────────────┐     ┌──┬──┬──┬──┐           ┌─────────────────┐
│             │     │00│10│20│30│           │T(0,0)  ...T(15,0)│
│   64 cols   │  →  ├──┼──┼──┼──┤     →     │  ...      ...   │
│   48 rows   │     │01│11│21│31│           │T(0,15)...T(15,15)│
│             │     ├──┼──┼──┼──┤           └─────────────────┘
└─────────────┘     │02│12│22│32│               256 threads
                    └──┴──┴──┴──┘
```

## Step 1: Complete the Exercise

Scale every element in a matrix: `output[row][col] = input[row][col] * scalar`

In [ ]:
%%writefile grid_2d.cu
/**
 * Exercise 04: 2D Grid Indexing
 */

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error: %s\n", cudaGetErrorString(err)); \
            exit(1); \
        } \
    } while(0)

// 2D kernel: scale each matrix element
__global__ void matrixScale(float *input, float *output, 
                            float scalar, int width, int height) {
    // Calculate 2D indices
    int col = blockIdx.x * blockDim.x + threadIdx.x;  // X → column
    int row = blockIdx.y * blockDim.y + threadIdx.y;  // Y → row
    
    // Bounds check (essential for partial blocks at edges!)
    if (col < width && row < height) {
        // Convert 2D to 1D index (row-major)
        int index = row * width + col;
        output[index] = input[index] * scalar;
    }
}

void printMatrix(float *m, int w, int h, const char *name) {
    printf("%s (showing 5x5 corner):\n", name);
    for (int r = 0; r < 5 && r < h; r++) {
        for (int c = 0; c < 5 && c < w; c++)
            printf("%5.1f ", m[r * w + c]);
        printf("...\n");
    }
    printf("...\n\n");
}

int main() {
    int width = 1024;
    int height = 768;
    float scalar = 2.5f;
    size_t size = width * height * sizeof(float);
    
    printf("Matrix: %d x %d, Scalar: %.1f\n", width, height, scalar);
    printf("Total elements: %d (%.2f MB)\n\n", width * height, size/1e6);
    
    // Host memory
    float *h_in = (float*)malloc(size);
    float *h_out = (float*)malloc(size);
    
    // Initialize
    for (int i = 0; i < width * height; i++)
        h_in[i] = (float)(i % 10);
    
    printMatrix(h_in, width, height, "Input");
    
    // Device memory
    float *d_in, *d_out;
    CUDA_CHECK(cudaMalloc(&d_in, size));
    CUDA_CHECK(cudaMalloc(&d_out, size));
    CUDA_CHECK(cudaMemcpy(d_in, h_in, size, cudaMemcpyHostToDevice));
    
    // 2D launch configuration
    dim3 threads(16, 16);  // 256 threads per block
    dim3 blocks(
        (width + threads.x - 1) / threads.x,
        (height + threads.y - 1) / threads.y
    );
    
    printf("Grid: %d x %d blocks\n", blocks.x, blocks.y);
    printf("Threads per block: %d x %d = %d\n", threads.x, threads.y, threads.x*threads.y);
    printf("Total threads: %d\n\n", blocks.x * blocks.y * threads.x * threads.y);
    
    // Launch kernel
    matrixScale<<<blocks, threads>>>(d_in, d_out, scalar, width, height);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());
    
    // Copy back
    CUDA_CHECK(cudaMemcpy(h_out, d_out, size, cudaMemcpyDeviceToHost));
    
    printMatrix(h_out, width, height, "Output");
    
    // Verify
    bool ok = true;
    for (int i = 0; i < width * height && ok; i++)
        if (fabsf(h_out[i] - h_in[i] * scalar) > 0.001f) ok = false;
    
    printf("%s\n", ok ? "✅ PASSED!" : "❌ FAILED!");
    
    CUDA_CHECK(cudaFree(d_in));
    CUDA_CHECK(cudaFree(d_out));
    free(h_in); free(h_out);
    
    return ok ? 0 : 1;
}

In [ ]:
!nvcc grid_2d.cu -o grid_2d && ./grid_2d

## 🧪 Experiment: Matrix Transpose

A more interesting 2D operation - swap rows and columns:

In [ ]:
%%writefile transpose.cu
#include <stdio.h>
#include <cuda_runtime.h>

// Transpose: output[col][row] = input[row][col]
__global__ void transpose(float *in, float *out, int width, int height) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (col < width && row < height) {
        int in_idx = row * width + col;       // Input:  [row][col]
        int out_idx = col * height + row;     // Output: [col][row]
        out[out_idx] = in[in_idx];
    }
}

void printMatrix(float *m, int w, int h, const char *name) {
    printf("%s (%dx%d):\n", name, w, h);
    for (int r = 0; r < h; r++) {
        for (int c = 0; c < w; c++)
            printf("%4.0f ", m[r * w + c]);
        printf("\n");
    }
    printf("\n");
}

int main() {
    int W = 4, H = 3;  // Small for visualization
    size_t size_in = W * H * sizeof(float);
    size_t size_out = H * W * sizeof(float);  // Transposed dimensions
    
    float *h_in = (float*)malloc(size_in);
    float *h_out = (float*)malloc(size_out);
    
    // Fill with identifiable values
    for (int r = 0; r < H; r++)
        for (int c = 0; c < W; c++)
            h_in[r * W + c] = r * 10 + c;  // e.g., 00, 01, 02, 03, 10, 11...
    
    printMatrix(h_in, W, H, "Input");
    
    float *d_in, *d_out;
    cudaMalloc(&d_in, size_in);
    cudaMalloc(&d_out, size_out);
    cudaMemcpy(d_in, h_in, size_in, cudaMemcpyHostToDevice);
    
    dim3 threads(16, 16);
    dim3 blocks((W + 15) / 16, (H + 15) / 16);
    transpose<<<blocks, threads>>>(d_in, d_out, W, H);
    cudaDeviceSynchronize();
    
    cudaMemcpy(h_out, d_out, size_out, cudaMemcpyDeviceToHost);
    
    // Note: transposed dimensions are (H x W)
    printMatrix(h_out, H, W, "Transposed");
    
    printf("Notice: rows became columns, columns became rows!\n");
    
    cudaFree(d_in); cudaFree(d_out);
    free(h_in); free(h_out);
    return 0;
}

In [ ]:
!nvcc transpose.cu -o transpose && ./transpose

---

## 🔄 Python Comparison (Optional)

In [ ]:
!pip install numba -q

from numba import cuda
import numpy as np

@cuda.jit
def matrix_scale_python(input, output, scalar):
    col, row = cuda.grid(2)  # 2D grid! Returns (x, y) tuple
    
    if row < input.shape[0] and col < input.shape[1]:
        output[row, col] = input[row, col] * scalar

# Test
height, width = 768, 1024
h_input = np.arange(height * width, dtype=np.float32).reshape(height, width) % 10
h_output = np.zeros_like(h_input)

d_input = cuda.to_device(h_input)
d_output = cuda.to_device(h_output)

# 2D configuration
threads = (16, 16)
blocks = ((width + 15) // 16, (height + 15) // 16)

matrix_scale_python[blocks, threads](d_input, d_output, 2.5)

result = d_output.copy_to_host()
print("Corner of result:")
print(result[:3, :5])
print(f"\n✅ Correct? {np.allclose(result, h_input * 2.5)}")

### C++ vs Python: 2D Indexing

| C++ | Python (Numba) |
|-----|----------------|
| `dim3 threads(16, 16)` | `threads = (16, 16)` |
| `dim3 blocks(x, y)` | `blocks = (x, y)` |
| `blockIdx.x * blockDim.x + threadIdx.x` | `cuda.grid(2)` returns tuple |
| Manual row-major indexing | NumPy 2D array indexing |

---

## ✅ Summary

You learned:
- **dim3** for 2D block/grid dimensions
- **col (x)** and **row (y)** from thread indices
- **Row-major layout**: `index = row * width + col`
- **Bounds checking** is essential for edge blocks

## ➡️ Next Steps

Continue with:
- [Day 2: Thread Indexing Notebook](../../../learning-path/week-01/day-2-thread-indexing.ipynb) - More patterns
- [Day 3: Memory Basics](../../../learning-path/week-01/day-3-memory-basics.ipynb) - Memory transfers